## Read in a segmentation dataset

In [2]:
from funlib.persistence import open_ds
import numpy as np

ds = open_ds(
    "/nrs/cellmap/zouinkhim/predictions/v21/2023_12_06_post_processed_2.n5",
    "mito_steps_3/Final_relabel",
)
ds.materialize()
data = ds.data.astype(
    np.uint16
)  # to save memory, might want to downsample for quicker on the fly mesh generation

## Display it in neuroglancer along with an image layer. Selecting an object will start an automatic mesh generation for that object

In [4]:
import neuroglancer
import socket

neuroglancer.set_server_bind_address("0.0.0.0")
viewer = neuroglancer.Viewer()
dimensions = neuroglancer.CoordinateSpace(
    names=["z", "y", "x"], units="nm", scales=ds.voxel_size
)

with viewer.txn() as state:
    state.dimensions = dimensions
    state.layers["raw"] = neuroglancer.ImageLayer(
        source="n5://https://cellmap-vm1.int.janelia.org/nrs/data/jrc_mus-liver-zon-1/jrc_mus-liver-zon-1.n5/em/fibsem-uint8"  # occasionally have issues here between http and https
    )

    state.layers["mito"] = neuroglancer.SegmentationLayer(
        # segments=[str(i) for i in np.unique(data[data > 0])], # this line will cause all objects to be selected and thus all meshes to be generated...will be slow if lots of high res meshes
        source=neuroglancer.LocalVolume(
            data=data,
            dimensions=neuroglancer.CoordinateSpace(
                names=["z", "y", "x"],
                units=["nm", "nm", "nm"],
                scales=ds.voxel_size,
                coordinate_arrays=[
                    None,
                    None,
                    None,
                ],
            ),
            voxel_offset=ds.roi.begin / ds.voxel_size,
        ),
    )
    print(viewer)

http://ackermand-ws2.hhmi.org:33173/v/195fa4b7154ddb319f1b7d23f302488845fdcd4e/
